<a href="https://colab.research.google.com/github/patipon-tech/Pretest-AI_Engineer/blob/main/Test_Ai_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# แปลงข้อมูลจาก 1M record เป็น 10 M reccord

In [10]:
import pandas as pd
import numpy as np

# 1️⃣ โหลดไฟล์ CSV เดิม
df = pd.read_csv("large_order_data_2024.csv")

# 2️⃣ ทำซ้ำ DataFrame เพื่อให้ได้ 10 ล้าน Record
repeat_times = 10
df_large = pd.concat([df.copy() for _ in range(repeat_times)], ignore_index=True)

# 3️⃣ เพิ่มความแปรปรวนในยอดขาย ±10% (ถ้าต้องการ)
np.random.seed(42)
df_large['amount'] = df_large['amount'] * (1 + np.random.uniform(-0.1, 0.1, size=len(df_large)))

# 4️⃣ ตรวจสอบจำนวน Record
print(df_large.shape)  # ควรเป็น (10_000_000, จำนวนคอลัมน์)

# 5️⃣ บันทึกเป็น CSV
df_large.to_csv("large_order_data_10M.csv", index=False)


(10000000, 8)


# Top 10 Spender

In [13]:
import pandas as pd

# 1️⃣ โหลดไฟล์ CSV ขนาดใหญ่
df = pd.read_csv("large_order_data_10M.csv")

# 2️⃣ แปลงคอลัมน์วันที่เป็น datetime
df['transaction_datetime'] = pd.to_datetime(df['transaction_datetime'])

# 3️⃣ สร้างคอลัมน์ year_month
df['year_month'] = df['transaction_datetime'].dt.to_period('M')

# 4️⃣ รวมยอดขายต่อ customer ต่อเดือน
monthly_sales = df.groupby(['year_month', 'customer_no'], as_index=False)['amount'].sum()

# 5️⃣ หา Top 10 spender ของแต่ละเดือน
top10_each_month = (
    monthly_sales
    .sort_values(['year_month', 'amount'], ascending=[True, False])
    .groupby('year_month', group_keys=False)
    .head(10)
)

# 6️⃣ เพิ่มคอลัมน์ rank ต่อเดือน
top10_each_month['rank'] = top10_each_month.groupby('year_month')['amount'].rank(method='first', ascending=False)

# 7️⃣ บันทึกผลลัพธ์เป็น CSV
top10_each_month.to_csv("top10_spender_each_month.csv", index=False)

print("Export CSV เรียบร้อย: top10_spender_each_month.csv")

Export CSV เรียบร้อย: top10_spender_each_month.csv
